## Programación Cuadrática
### Estimación de Pesos de un Indice Bursátil
Este cuaderno muestra un ejemplo de como utilizar una
optimización de mínimos cuadrados con restricciones para estimar
los pesos de los componentes del IBEX35

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [ ]:
import pickle
with open('../data/stock_data.pkl','rb') as f:
    stock_data = pickle.load(f)
with open('../data/benchmark.pkl','rb') as f:
    benchmark_data = pickle.load(f)
    

In [ ]:
stock_close = pd.DataFrame({
    ticker: df.close for ticker, df in stock_data.items()
})

In [ ]:
stock_close.head()

Trabajaremos con una ventana de 35 dias para simular una matriz cuadrada
de 35x35 que sería suficiente para resolver el problema de minimos cuadrados
de forma algebraica

In [ ]:
win_data = stock_close.loc['2020-07-06':'2020-08-21']
win_data = win_data.dropna(axis=1)

In [ ]:
print(win_data.shape)
win_data.head()

In [ ]:
ibex_close = benchmark_data['ibex_div'].close
win_ibex = ibex_close.reindex(win_data.index)
win_ibex.head()

Convertimos los precios a precios relativos respecto al primer
día de la ventana 

In [ ]:
prices_norm = win_data.apply(lambda s: s/s.iloc[0])
ibex_norm = win_ibex/win_ibex.iloc[0]

In [ ]:
prices_norm.head()

#### Las variables de decisión
- los pesos de cada componente
- el término de error para cada día

In [ ]:
window = 35
n_stocks = prices_norm.shape[1]
var_w = cp.Variable(n_stocks)
var_epsilon = cp.Variable(window)

#### Restricciones

- Para cada día queremos que el precio relativo por el peso de cada componente se parezca al valor del precio relativo del indice 
- La suma de los pesos debe ser 1
- Podemos limitar los pesos máximos y minimos. Como requerimiento debemos al menos asegurar que los pesos son positivos 

In [ ]:
c_changes = []
for i in range(window):
    i_constraint = cp.sum(cp.multiply(var_w, prices_norm.iloc[i].values)) - ibex_norm[i] == var_epsilon[i] 
    c_changes.append(i_constraint)

constraints = [
    var_w <= 0.25, 
    var_w >= 0.003, 
    cp.sum(var_w) <= 1.0,
    cp.sum(var_w) >= 1.0
]        
constraints.extend(c_changes)

#### Función objetivo
Minimizar la suma de los cuadrados de los errores diarios respecto al indice

In [ ]:
objective = cp.Minimize( cp.sum_squares(var_epsilon))

Resolvemos el problema y almacenamos el resultado de los pesos

In [ ]:
prob = cp.Problem(objective, constraints)
prob.solve()

In [ ]:
errors = pd.Series(var_epsilon.value)
index_weights = pd.Series(var_w.value, index=prices_norm.columns)

Las variables de decisión estan referenciando a los pesos del primer
día.  Sacamos los pesos al último día de la serie

In [ ]:
index_weights.sum()

In [ ]:
index_weights.sort_values(ascending=False)

Miramos los errores obtenidos por día para verificar que no existe ningún patrón relevante

In [ ]:
errors.plot()